In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = None
train_size  = 45000
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 256
epochs = 200

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[train_size:]
y_valid = y_train_all[train_size:]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(
    100000
).batch(
    batch_size
).prefetch(10)

In [9]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)

In [10]:
layers = tf.keras.layers

In [11]:
class BasicBlock(tf.keras.layers.Layer):

    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        if stride != 1:
            self.downsample = tf.keras.Sequential()
            self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                       kernel_size=(1, 1),
                                                       strides=stride))
            self.downsample.add(tf.keras.layers.BatchNormalization())
        else:
            self.downsample = lambda x: x

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output

def make_basic_block_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BasicBlock(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BasicBlock(filter_num, stride=1))

    return res_block

In [12]:
class ResNetTypeI(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeI, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(filters=16,
                                            kernel_size=(3, 3),
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.layer1 = make_basic_block_layer(filter_num=16,
                                             blocks=layer_params[0])
        self.layer2 = make_basic_block_layer(filter_num=32,
                                             blocks=layer_params[1],
                                             stride=2)
        self.layer3 = make_basic_block_layer(filter_num=64,
                                             blocks=layer_params[2],
                                             stride=2)
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(10)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output

In [13]:
model = ResNetTypeI([5, 5, 5])

In [14]:
model.build(input_shape=(batch_size,*image_shape))

In [15]:
model.summary()

Model: "res_net_type_i"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  448       
_________________________________________________________________
batch_normalization (BatchNo multiple                  64        
_________________________________________________________________
sequential (Sequential)      multiple                  23840     
_________________________________________________________________
sequential_1 (Sequential)    multiple                  89824     
_________________________________________________________________
sequential_3 (Sequential)    multiple                  355776    
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple               

In [16]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [17]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [19]:
# model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

In [20]:
model.load_weights('./model_weights/resnet32_train_without_DA=all_ce.h5')

In [21]:
model.evaluate(x=x_test, y=y_test_all, verbose=0)

[1.2938404957294465, 0.7832]

In [23]:
file_list = ['cifar-fgsm-eps-0.03-time-500000.npy',
             'cifar-fgsm-eps-0.03-time-100000.npy',
             'cifar-fgsm-eps-0.03-time-1000000.npy',
             'cifar-fgsm-eps-0.03-time-2300000.npy',
             './npy/batch_NTK_simple.npy',
             './npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy',
             './npy/batch_NTK_cnn19.npy',
             './npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy',
             "./npy/batch_NTK_cnn19_decrease_variance.npy",
             "./npy/batch_NTK_cnn19_increase_variance.npy",
             "./npy/batch_NTK_simple_decrease_variance.npy",
             "./npy/batch_NTK_simple_increase_variance.npy"]

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp[:512], y_test[:512], verbose=0)
    print(result)

evaluating file: cifar-fgsm-eps-0.03-time-500000.npy
[1.918043628334999, 0.68359375]
evaluating file: cifar-fgsm-eps-0.03-time-100000.npy
[1.9162272736430168, 0.68359375]
evaluating file: cifar-fgsm-eps-0.03-time-1000000.npy
[1.918043628334999, 0.68359375]
evaluating file: cifar-fgsm-eps-0.03-time-2300000.npy
[1.918043628334999, 0.68359375]
evaluating file: ./npy/batch_NTK_simple.npy
[1.918043628334999, 0.68359375]
evaluating file: ./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy
[2.1466585397720337, 0.6816406]
evaluating file: ./npy/batch_NTK_cnn19.npy
[1.8007595464587212, 0.73046875]
evaluating file: ./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy
[1.9180336445569992, 0.7285156]
evaluating file: ./npy/batch_NTK_cnn19_decrease_variance.npy
[1.7933332100510597, 0.734375]
evaluating file: ./npy/batch_NTK_cnn19_increase_variance.npy
[1.8046077489852905, 0.73046875]
evaluating file: ./npy/batch_NTK_simple_decrease_variance.npy
[1.8823599740862846, 0.68359375]
eval

In [22]:
file_list = ['batch_NTK_simple_decrease_variance.npy',
             'batch_NTK_simple_increase_variance.npy',
             'batch_NTK_CNN10_decrease_variance.npy',
             'batch_NTK_CNN10_increase_variance.npy']

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp, y_test[:2048], verbose=0)
    print(result)

evaluating file: batch_NTK_simple_decrease_variance.npy
[1.8453136654570699, 0.6958008]
evaluating file: batch_NTK_simple_increase_variance.npy
[1.836729809641838, 0.7026367]
evaluating file: batch_NTK_CNN10_decrease_variance.npy
[1.7003457713872194, 0.7324219]
evaluating file: batch_NTK_CNN10_increase_variance.npy
[1.7083656825125217, 0.72753906]


In [ ]:
# with pooling, acc can reach 72.2%
# w.o. pooling, acc is at most 67%

In [22]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128], verbose=0)

==========NTK============


[1.4168568849563599, 0.734375]

In [23]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========CE============


[0.7829372922424227, 0.8383789]

In [24]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========MSE============


[0.9031526042381302, 0.8071289]

In [26]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[1.589853584766388, 0.7109375]

In [23]:
tmp = onp.load('./batch_NTK_cnn_19.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[1.7031463384628296, 0.72802734375]

In [26]:
tmp = onp.load('./batch_NTK_simple.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[1.8305855989456177, 0.70068359375]

In [27]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy')
print("==========small============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========small============


[2.133112907409668, 0.67529296875]

In [28]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[1.746604561805725, 0.72314453125]